# 07_metrics_lab — 분류 모델 평가 지표

오늘 배울 것
1) **정확도(Accuracy)의 함정**: 왜 때로는 위험한가 (불균형 데이터에서 착시)
2) **혼동행렬(Confusion Matrix)**: 무엇을 많이 틀렸는지 보기
3) **정밀도(Precision), 재현율(Recall), F1**: 의미와 해석
4) **임곗값(Threshold)**: 바꾸면 지표가 어떻게 달라지는지 (표/그래프)
5) **ROC 곡선과 AUC, PR 곡선(AP)**: 전체 임곗값에서의 분류력 요약

### 오늘 쓸 지표
- **정확도(Accuracy)** = (맞춘 개수 / 전체). 직관적이지만 **불균형**에서 착시 가능
- **혼동행렬(Confusion Matrix)** = 무엇을 얼마나 틀렸는지(TP/FP/FN/TN)
- **정밀도(Precision)** = '양성이라고 예측'한 것 중 실제 양성 비율 (오탐↓)
- **재현율(Recall)** = 실제 양성 중 놓치지 않고 찾은 비율 (누락↓)
- **F1 점수(F1 Score)** = 정밀도·재현율의 **균형**
- **ROC-AUC** = 임곗값 전 범위 분류력 요약(왼쪽 위로 갈수록 좋음)
- **PR-AP** = 불균형에서 더 직관적(양성 비율 기준선 위여야 의미 있음)

### 핵심 메시지
- 정확도 하나만 보지 않는다.
- 혼동행렬, Precision, Recall, F1, ROC-AUC, PR-AP까지 함께 본다.
- 임곗값(Threshold)을 바꾼다는 건 곧 정책을 선택하는 것이다.
- 스팸메일 필터라면? 정밀도(Precision)를 우선할 수 있음.
- 암 진단이라면? 재현율(Recall)을 최우선으로 봐야 함.

In [66]:
import sklearn, numpy as np, pandas as pd
import plotly.express as px
import plotly.graph_objects as go
print("sklearn:", sklearn.__version__)
print("numpy  :", np.__version__)
print("pandas :", pd.__version__)

sklearn: 1.7.2
numpy  : 2.3.3
pandas : 2.3.2


## Titanic 데이터셋 소개
- 실제 타이타닉호 생존자 데이터를 정리한 유명한 공개 데이터셋
- 목표: 승객의 나이, 성별, 선실 등급 등을 보고 **생존 여부(0=사망, 1=생존)**를 예측
- 주요 컬럼:
  - `Pclass`: 선실 등급 (1=일등석, 2=이등석, 3=삼등석)
  - `Sex`: 성별
  - `Age`: 나이
  - `SibSp`: 동반한 형제/배우자 수
  - `Parch`: 동반한 부모/자녀 수
  - `Fare`: 운임 요금
  - `Embarked`: 출항 항구 (C, Q, S)
- **Target(정답)**: `Survived` (1=생존, 0=사망)

## 1. Accuracy(정확도) — Titanic Dummy 예제로 착시 보기
- Titanic 데이터를 전처리하고 Dummy 분류기로 정확도를 출력
- Dummy = 매우 단순한 규칙(성별만 보고 생존 예측).
- **주의**: 전처리는 데모용이라 누수 위험이 있음

In [41]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def fillna(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna(0)
    return df

def drop_features(df):
    return df.drop(['PassengerId','Name','Ticket'], axis=1)

def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    for feature in ['Cabin','Sex','Embarked']:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

def transform_features(df):
    return format_features(drop_features(fillna(df)))

titanic_df = pd.read_csv('../day_01/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = transform_features(titanic_df.drop('Survived', axis=1))

### Dummy Classifier (항상 같은 규칙만 적용)
- 성별이 남성(1)이면 무조건 0(사망), 아니면 1(생존)으로 예측.
- 정확도 출력

In [42]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class MyDummyClassifier(BaseEstimator):
    def fit(self, X, y=None): return self
    def predict(self, X):
        return np.where(X['Sex'].values==1, 0, 1).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, stratify=y_titanic_df, random_state=0
)

pred = MyDummyClassifier().fit(X_train, y_train).predict(X_test)
print('Dummy Classifier 정확도:', accuracy_score(y_test, pred))

Dummy Classifier 정확도: 0.776536312849162


### 불균형 데이터란?
- 한 클래스가 **극소수**인 상황(예: 사기, 암 환자, 스팸)
- 이때 Accuracy는 쉽게 높아지므로 **지표를 다양하게** 본다

## 2. 불균형 데이터 예 — digits에서 '4' vs 나머지
- 목표: **불균형 데이터**에서 "전부 0만 예측해도 정확도 높음" 착시를 보여줌

In [43]:
from sklearn.datasets import load_digits

digits = load_digits()
y = (digits.target == 4).astype(int)  # 4이면 1, 아니면 0

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, y, test_size=0.2, stratify=y, random_state=11
)

print(f"테스트셋 클래스 비율:\n {pd.Series(y_test).value_counts(normalize=True)}")

테스트셋 클래스 비율:
 0    0.9
1    0.1
Name: proportion, dtype: float64


In [44]:
import plotly.express as px

dist_df = pd.DataFrame({"label": y_test})
fig = px.histogram(dist_df, x="label", color="label", text_auto=True,
                   title="digits (test) — Class Distribution (4 vs others)",
                   category_orders={"label":[0,1]},
                   template="plotly_dark")
fig.update_xaxes(title="Label (0=others, 1=four)")
fig.update_yaxes(title="Count")
fig.show()

### Dummy(All-0) 결과: Accuracy는 높지만, 양성을 전혀 못 맞춤

In [45]:
from sklearn.metrics import precision_score, recall_score

class FakeClassifier(BaseEstimator):
    def fit(self, X, y=None): return self
    # 모든 입력에 대한 것들 중 무조건 0만
    def predict(self, X): return np.zeros(len(X), dtype=int)

fakepred = FakeClassifier().fit(X_train, y_train).predict(X_test)

# 전체 중에 맞춘 비율
print(f"Accuracy: {accuracy_score(y_test, fakepred):.3f}")
# 모델이 양성(1)이라고 한 것 중에서 진짜 양성 비율
print("Precision:", precision_score(y_test, fakepred, zero_division=0))
# 실제 양성(1) 중에서 모델이 제대로 잡은 비율
print("Recall   :", recall_score(y_test, fakepred, zero_division=0))

Accuracy: 0.900
Precision: 0.0
Recall   : 0.0


### 혼동행렬 읽는 법
- **TN**: 음성인데 음성으로 맞춤
- **FP**: 음성인데 양성으로 틀림(오탐)
- **FN**: 양성인데 음성으로 틀림(누락)
- **TP**: 양성인데 양성으로 맞춤
> Dummy(all-zero)는 FN이 가득 → 양성을 전혀 못 잡는다(Recall≈0)

## Confusion Matrix와 성적표
- 정확도 외에 **무엇을 많이 틀렸는지** 보여줌

In [46]:
from sklearn.metrics import confusion_matrix, classification_report
import plotly.figure_factory as ff
import numpy as np

cm = confusion_matrix(y_test, fakepred, labels=[0, 1])  # 레이블 고정 추천

ann = np.array([
    [f"TN={cm[0,0]}", f"FP={cm[0,1]}"],
    [f"FN={cm[1,0]}", f"TP={cm[1,1]}"]
])

fig = ff.create_annotated_heatmap(
    z=cm,
    x=["예측=0", "예측=1"],
    y=["실제=0", "실제=1"],
    annotation_text=ann,
    colorscale="Blues", showscale=True
)
fig.update_layout(
    title="Confusion Matrix (Dummy: all-zero)",
    xaxis=dict(title="Predicted"),
    yaxis=dict(title="Actual"),
    template="plotly_dark"
)
fig.show()

# classification_report 출력
print(classification_report(y_test, fakepred, digits=3, zero_division=0))

              precision    recall  f1-score   support

           0      0.900     1.000     0.947       324
           1      0.000     0.000     0.000        36

    accuracy                          0.900       360
   macro avg      0.450     0.500     0.474       360
weighted avg      0.810     0.900     0.853       360



### 혼동행렬 읽는 법 (TN/FP/FN/TP)
- **TN**: 음성인데 음성으로 맞춤
- **FP**: 음성인데 양성으로 잘못 예측
- **FN**: 양성인데 음성으로 잘못 예측
- **TP**: 양성인데 양성으로 맞춤

In [47]:
import pandas as pd
import plotly.graph_objects as go

# 가정: 클래스별 F1 점수
scores = {"class": [0, 1], "support": [90, 10], "f1": [0.94, 0.00]}
df = pd.DataFrame(scores)

# macro avg: 단순 평균
macro_avg = df["f1"].mean()

# weighted avg: support(표본 수) 가중 평균
weighted_avg = (df["f1"] * df["support"]).sum() / df["support"].sum()

# Plotly Table 시각화
fig = go.Figure(data=[go.Table(
    header=dict(values=["Class", "Support", "F1 Score"],
                fill_color="purple", align="center"),
    cells=dict(values=[df["class"], df["support"], df["f1"].round(3)],
               fill_color="black", align="center")
)])

fig.update_layout(title="클래스별 F1 Score", template="plotly_dark")
fig.show()

print(f"Macro avg (단순 평균): {macro_avg:.3f}")
print(f"Weighted avg (가중 평균): {weighted_avg:.3f}")

Macro avg (단순 평균): 0.470
Weighted avg (가중 평균): 0.846


## 핵심 지표 공식
- **Accuracy** = (TP + TN) / (TP + FP + FN + TN)
- **Precision** = TP / (TP + FP)
- **Recall**    = TP / (TP + FN)
- **F1**        = 2 · (Precision · Recall) / (Precision + Recall)

설명:
- 정밀도(Precision): '양성이라 예측'한 것 중 실제 양성 비율 (오탐↓)
- 재현율(Recall): 실제 양성 중 놓치지 않고 찾은 비율 (누락↓)

## 업무에 따른 중요도
- 정밀도: 실제 Negative 음성인 데이터 예측을 Positive로 판단하면 안된느 경우: 스팸메일
- 재현율: 실제 Positive 양성인 데이터 예측을 Negative로 판단하면 안되는 경우: 금융 사기, 암 진단

## 3. 기준선 모델: Logistic Regression
- Dummy와 비교할 실제 모델 학습

In [48]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=5000, random_state=42).fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Accuracy:", accuracy_score(y_test, y_pred_lr))

Logistic Accuracy: 1.0


In [49]:
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support

print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_lr))

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred_lr, average='macro', zero_division=0)
prec_weight, rec_weight, f1_weight, _ = precision_recall_fscore_support(y_test, y_pred_lr, average='weighted', zero_division=0)

print(f"Macro     P:{prec_macro:.3f} R:{rec_macro:.3f} F1:{f1_macro:.3f}")
print(f"Weighted  P:{prec_weight:.3f} R:{rec_weight:.3f} F1:{f1_weight:.3f}")


Balanced Accuracy: 1.0
Macro     P:1.000 R:1.000 F1:1.000
Weighted  P:1.000 R:1.000 F1:1.000


In [50]:
# import plotly.graph_objects as go
#
# acc = accuracy_score(y_test, y_pred_lr)
# bal_acc = balanced_accuracy_score(y_test, y_pred_lr)
#
# prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
#     y_test, y_pred_lr, average='macro', zero_division=0
# )
# prec_weight, rec_weight, f1_weight, _ = precision_recall_fscore_support(
#     y_test, y_pred_lr, average='weighted', zero_division=0
# )
#
# metrics = ["Accuracy", "Balanced Accuracy", "Macro F1", "Weighted F1"]
# values = [acc, bal_acc, f1_macro, f1_weight]
#
# fig = go.Figure(data=[go.Table(
#     header=dict(values=["Metric", "Value"], fill_color="purple", align="center"),
#     cells=dict(values=[metrics, [f"{v:.3f}" for v in values]],
#                fill_color="black", align="center")
# )])
#
# fig.update_layout(title="Logistic Regression 성능 지표 요약", template="plotly_dark")
# fig.show()

### 평균 방식 요약
- **Balanced Accuracy** = 클래스별 Recall 평균(불균형에서 공정)
- **Macro 평균** = 클래스별 지표 **균등 평균**(소수 클래스 반영↑)
- **Weighted 평균** = 표본 수 가중 평균(전체 분포 반영)

### 임곗값(Threshold) = 정책 선택
- 임곗값 낮춤 → **Recall↑ Precision↓** (더 많이 양성으로 잡음, 오탐↑)
- 임곗값 높임 → **Precision↑ Recall↓** (확실할 때만 양성, 누락↑)
- **F1 최대** 또는 정책 제약(예: Recall≥90%)으로 임곗값을 결정

### 임곗값(Threshold) 실험
- 확률 출력 → 기준선 0.5 대신 다양한 임곗값으로 Precision/Recall/F1/Accuracy 확인

In [67]:
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

digits = load_digits()
y = (digits.target == 7).astype(int)

X = PCA(n_components=10, random_state=42).fit_transform(digits.data)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=11
)

lr = LogisticRegression(max_iter=5000, C=0.1, random_state=42)
lr.fit(X_train, y_train)

y_proba_lr = lr.predict_proba(X_test)[:, 1]
thr_list = np.round(np.arange(0.1, 1.0, 0.1), 2)

rows = []
for thr in thr_list:
    y_hat = (y_proba_lr >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0),
        accuracy_score(y_test, y_hat)
    ])

thr_table = pd.DataFrame(rows, columns=["threshold","precision","recall","f1","accuracy"])
display(thr_table)

# 해석을 도와주는 2줄: F1 최대 / Recall≥0.90 최소 임곗값
best = thr_table.iloc[thr_table['f1'].idxmax()]
print(f"[F1 최대] thr={best['threshold']:.2f}, P={best['precision']:.3f}, R={best['recall']:.3f}, F1={best['f1']:.3f}")

target_recall = 0.90
cand = thr_table[thr_table['recall'] >= target_recall].sort_values('threshold')
print(f"[Recall≥{target_recall:.2f}]",
      "없음" if cand.empty else f"thr={cand.iloc[0]['threshold']:.2f}, P={cand.iloc[0]['precision']:.3f}, F1={cand.iloc[0]['f1']:.3f}")

,threshold,precision,recall,f1,accuracy
0,0.1,0.708333,0.944444,0.809524,0.955556
1,0.2,0.772727,0.944444,0.850000,0.966667
2,0.3,0.809524,0.944444,0.871795,0.972222
3,0.4,0.829268,0.944444,0.883117,0.975000
4,0.5,0.850000,0.944444,0.894737,0.977778
5,0.6,0.864865,0.888889,0.876712,0.975000
6,0.7,0.911765,0.861111,0.885714,0.977778
7,0.8,0.967742,0.833333,0.895522,0.980556
8,0.9,0.961538,0.694444,0.806452,0.966667


[F1 최대] thr=0.80, P=0.968, R=0.833, F1=0.896
[Recall≥0.90] thr=0.10, P=0.708, F1=0.810


In [59]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="cyan")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="magenta")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="yellow")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                  template="plotly_dark")
fig.show()

## Precision/Recall vs Threshold (임곗값 변화)
- Recall↑ ↔ Precision↓ (트레이드오프)

In [14]:
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.graph_objects as go

fpr, tpr, thr = roc_curve(y_test, y_proba_lr)

auc = roc_auc_score(y_test, y_proba_lr)

fig_roc = go.Figure()

# ROC 곡선
fig_roc.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode="lines", name=f"ROC (AUC={auc:.3f})",
    line=dict(color="cyan", width=3)
))

# 랜덤 기준선 (y=x, AUC=0.5)
fig_roc.add_trace(go.Scatter(
    x=[0,1], y=[0,1],
    mode="lines", name="Random",
    line=dict(dash="dash", color="grey")
))

fig_roc.update_layout(
    title="ROC Curve",
    xaxis_title="FPR (거짓 양성 비율)",
    yaxis_title="TPR (재현율/Recall)",
    template="plotly_dark",
    xaxis=dict(range=[0,1]),
    yaxis=dict(range=[0,1])
)

fig_roc.show()

### ROC vs PR: 어떻게 쓸까?
- **ROC-AUC**: 임곗값 전 범위에서 분류력 요약(0.5 랜덤, 1.0 완벽)
- **PR-AP**: **불균형** 데이터에서 더 직관적(양성 비율 기준선 위 유지)
> 가장 안전한 방법: **두 그래프를 함께 보는 습관**

## 4. ROC & AUC, PR & AP
- ROC: 전체 임곗값에서 거짓 양성/진짜 양성 비율
- PR: 불균형 데이터에서 더 직관적

## PR

In [63]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import plotly.graph_objects as go

# 1) PR Curve 계산
prec, rec, thr = precision_recall_curve(y_test, y_proba_lr)
ap = average_precision_score(y_test, y_proba_lr)

# 2) 양성 비율 (baseline)
pos_rate = y_test.mean()

# 3) Plotly 시각화
fig_pr = go.Figure()

# PR 곡선
fig_pr.add_trace(go.Scatter(
    x=rec, y=prec,
    mode="lines", name=f"PR (AP={ap:.3f})",
    line=dict(color="magenta", width=3)
))

# 기준선 (양성 비율)
fig_pr.add_trace(go.Scatter(
    x=[0,1], y=[pos_rate, pos_rate],
    mode="lines", name=f"Baseline pos rate={pos_rate:.2f}",
    line=dict(dash="dash", color="grey")
))

fig_pr.update_layout(
    title="Precision-Recall Curve",
    xaxis_title="Recall",
    yaxis_title="Precision",
    template="plotly_dark",
    yaxis=dict(range=[0,1]),
    xaxis=dict(range=[0,1])
)

fig_pr.show()

## ROC

In [64]:
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.graph_objects as go

fpr, tpr, thr = roc_curve(y_test, y_proba_lr)
auc = roc_auc_score(y_test, y_proba_lr)

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode="lines",
                             name=f"ROC (AUC={auc:.3f})",
                             line=dict(color="cyan", width=3)))
fig_roc.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines",
                             name="Random", line=dict(dash="dash", color="grey")))
fig_roc.update_layout(title="ROC Curve",
                      xaxis_title="FPR (거짓 양성 비율)",
                      yaxis_title="TPR (재현율/Recall)",
                      template="plotly_dark", xaxis=dict(range=[0,1]), yaxis=dict(range=[0,1]))
fig_roc.show()

## 요약
- **정확도만** 보면 위험하다 (특히 불균형 데이터).
- Confusion Matrix, Precision, Recall, F1으로 보완해야 함
- 임곗값을 조절하면 Precision-Recall 균형을 상황에 맞게 바꿀 수 있음
- ROC-AUC, PR-AP는 전체 성능 요약 지표로 중요

In [17]:
import plotly.graph_objects as go

# ROC vs PR 비교 내용
metrics = ["ROC Curve", "PR Curve"]
x_axis = [
    "FPR (False Positive Rate)",
    "Recall (재현율)"
]
y_axis = [
    "TPR (True Positive Rate, 재현율)",
    "Precision (정밀도)"
]
interpret = [
    "곡선이 왼쪽 위로 갈수록 좋음 (AUC↑)",
    "곡선이 오른쪽 위로 갈수록 좋음 (AP↑)"
]
when_use = [
    "클래스가 비교적 균형 있을 때 전체 성능 요약",
    "불균형 데이터에서 더 직관적"
]

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["지표", "X축", "Y축", "해석", "언제 유용한가"],
        fill_color="purple", align="center", font=dict(color="white")
    ),
    cells=dict(
        values=[metrics, x_axis, y_axis, interpret, when_use],
        fill_color=[["black","black"]],
        align="center",
        font=dict(color="white")
    )
)])

fig.update_layout(
    title="ROC vs PR Curve 비교",
    template="plotly_dark"
)

fig.show()